In [1]:
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import keras
import numpy as np
import os
import re
import tensorflow as tf

Using TensorFlow backend.


In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import math
from sklearn import metrics 
import seaborn as sns

import pandas as pd
import cv2
import glob

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [44]:
base_path="/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3"
model_set = {"inceptionv3","resnet","MobileNetV2"}
selected_model = "inceptionv3" # should be equal to the name of hdf5 model_name should be IN model_set

In [5]:
# .hdf5 file format is exported_model_{{model name}}.hdf5
model=tf.keras.models.load_model(os.path.join(base_path,"exported_model_{}.hdf5".format(selected_model)))

In [6]:
data_dir=os.path.join(base_path,"Kaggle_Dataset")
test_dir=os.path.join(data_dir,"imgs/test")
train_dir=os.path.join(data_dir,"imgs/train")
target_size=(150,150)

In [7]:
train_datagen=ImageDataGenerator(rescale=1./255)

test_datagen=ImageDataGenerator(rescale=1./255)

# train_datagen=ImageDataGenerator(rescale=1./255,
#                                  )

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=target_size,
                                                  batch_size=60,
                                                  class_mode="categorical"
                                                  )

# test_generator=test_datagen.flow_from_dataframe(test_df,
#                                                   target_size=target_size,
#                                                   batch_size=1,
#                                                   class_mode="categorical",
#                                                   shuffle=False
#                                                   )

Found 22424 images belonging to 10 classes.


In [8]:
train_file_list=glob.iglob(os.path.join(train_dir,"**/*.jpg"),recursive=True)

In [9]:
test_file_list=glob.iglob(os.path.join(test_dir,"*.jpg"),recursive=True)


In [10]:
files=[]
for file in test_file_list:
  files.append(file)

Streaming output truncated to the last 5000 lines.
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14537.jpg
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14596.jpg
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14445.jpg
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14809.jpg
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14710.jpg
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14454.jpg
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14342.jpg
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14775.jpg
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14729.jpg
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14346.jpg
/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_14

In [12]:
sorted_files=np.sort(files)

In [16]:
sorted_files

array(['/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_1.jpg',
       '/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_10.jpg',
       '/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_100.jpg',
       ...,
       '/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_99996.jpg',
       '/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_99998.jpg',
       '/content/gdrive/My Drive/42028-DL-CNN-2020/Ass3/Kaggle_Dataset/imgs/test/img_99999.jpg'],
      dtype='<U87')

In [42]:
header=["fileName","label"]
test_data=[[file,"0"] for file in sorted_files]
test_df=pd.DataFrame(test_data,columns=header)

In [23]:
test_generator=test_datagen.flow_from_dataframe(test_df,
                                                x_col="fileName",
                                                y_col="label",
                                                target_size=target_size,
                                                batch_size=1,
                                                class_mode="categorical",
                                                shuffle=False
                                                )

Found 9 validated image filenames belonging to 3 classes.


In [26]:
#predict by filename
predicted_result=model.predict_generator(test_generator)
predictions=[np.argmax(row) for row in predicted_result]
test_labels=test_generator.classes
predicted_result

array([[3.0574293e-03, 8.0357566e-03, 5.5912718e-05, 9.8245066e-01,
        4.6465276e-03, 3.2869281e-04, 7.1373652e-05, 6.9041771e-06,
        8.4104562e-05, 1.2626330e-03],
       [2.7121500e-06, 5.7376863e-04, 1.4278927e-05, 1.0515754e-09,
        1.0442700e-09, 3.0916556e-08, 1.1631713e-05, 9.9899262e-01,
        1.4074052e-05, 3.9086223e-04],
       [1.7367620e-03, 2.0068551e-04, 5.9317181e-06, 5.0108785e-05,
        5.8749167e-05, 6.5886771e-04, 1.3055114e-05, 4.4233061e-04,
        2.4602251e-04, 9.9658751e-01],
       [3.0574293e-03, 8.0357566e-03, 5.5912718e-05, 9.8245066e-01,
        4.6465276e-03, 3.2869281e-04, 7.1373652e-05, 6.9041771e-06,
        8.4104562e-05, 1.2626330e-03],
       [2.7121500e-06, 5.7376863e-04, 1.4278927e-05, 1.0515754e-09,
        1.0442700e-09, 3.0916556e-08, 1.1631713e-05, 9.9899262e-01,
        1.4074052e-05, 3.9086223e-04],
       [1.7367620e-03, 2.0068551e-04, 5.9317181e-06, 5.0108785e-05,
        5.8749167e-05, 6.5886771e-04, 1.3055114e-05, 4.42

In [29]:
#build first column
img_list = [ file.split("/")[-1] for file in sorted_files]
img_col = np.array(img_list).reshape(-1,1)

In [46]:
#generate the submission for kaggle
table = np.concatenate((img_col,predicted_result),axis=1)
headers = ["img","c0","c1","c2","c3","c4","c5","c6","c7","c8","c9"]
df = pd.DataFrame(table,columns=headers)
df.to_csv(os.path.join(base_path,"{}_result.csv".format(selected_model)),index=False)